In [1]:
from sklearn.datasets import load_digits, load_wine
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)


In [2]:
from memento import Config, Context, Memento

# Setup Configuration Matrix

In [3]:
matrix = {
    "parameters": {
        "dataset": [load_digits, load_wine],
        "preprocessing": [MinMaxScaler(), StandardScaler()],
        "classifier": [AdaBoostClassifier, RandomForestClassifier, SVC],
    },
    "settings": {
        "n_fold": 20,
    },
    "exclude": [
        {"dataset": load_digits, "classifier": SVC}
    ],
}

# Setup Experiment Function

In [4]:
def experiment(context: Context, config: Config):
    data = config.dataset()
    model = config.classifier()

    cv = config.settings["n_fold"]

    pipeline = make_pipeline(config.preprocessing, model)
    results = cross_val_score(pipeline, data.data, data.target, cv=cv)
    context.checkpoint(results)
    return results.mean() * 100

# Run Experiments & Get Results

In [5]:
results = Memento(experiment).run(matrix, dry_run=True)

INFO:memento.memento:Running configurations:
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'MinMaxScaler()', 'classifier': 'AdaBoostClassifier'}
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'MinMaxScaler()', 'classifier': 'RandomForestClassifier'}
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'StandardScaler()', 'classifier': 'AdaBoostClassifier'}
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'StandardScaler()', 'classifier': 'RandomForestClassifier'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'MinMaxScaler()', 'classifier': 'AdaBoostClassifier'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'MinMaxScaler()', 'classifier': 'RandomForestClassifier'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'MinMaxScaler()', 'classifier': 'SVC'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'StandardScaler()', 'classifier': 'AdaBoostClass

In [6]:
results = Memento(experiment).run(matrix)

INFO:memento.memento:Running configurations:
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'MinMaxScaler()', 'classifier': 'AdaBoostClassifier'}
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'MinMaxScaler()', 'classifier': 'RandomForestClassifier'}
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'StandardScaler()', 'classifier': 'AdaBoostClassifier'}
INFO:memento.memento:  {'dataset': 'load_digits', 'preprocessing': 'StandardScaler()', 'classifier': 'RandomForestClassifier'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'MinMaxScaler()', 'classifier': 'AdaBoostClassifier'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'MinMaxScaler()', 'classifier': 'RandomForestClassifier'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'MinMaxScaler()', 'classifier': 'SVC'}
INFO:memento.memento:  {'dataset': 'load_wine', 'preprocessing': 'StandardScaler()', 'classifier': 'AdaBoostClass

In [7]:
def resultsToTable(results):
    res = [[round(r.inner, 2), r.config.dataset.__name__, r.config.preprocessing.__class__.__name__, r.config.classifier.__name__] for r in results]
    return pd.DataFrame(res, columns=['Accuracy', 'Dataset', 'Preprocessing', 'Classifier'])

In [8]:
resultsToTable(results)

,Accuracy,Dataset,Preprocessing,Classifier
0,27.10,load_digits,MinMaxScaler,AdaBoostClassifier
1,96.66,load_digits,MinMaxScaler,RandomForestClassifier
2,27.15,load_digits,StandardScaler,AdaBoostClassifier
3,96.77,load_digits,StandardScaler,RandomForestClassifier
4,88.89,load_wine,MinMaxScaler,AdaBoostClassifier
5,97.78,load_wine,MinMaxScaler,RandomForestClassifier
6,98.89,load_wine,MinMaxScaler,SVC
7,88.33,load_wine,StandardScaler,AdaBoostClassifier
8,97.78,load_wine,StandardScaler,RandomForestClassifier
9,98.33,load_wine,StandardScaler,SVC
